In [1]:
import sys
import random
import pickle
import argparse
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from pprint import pprint

sys.path.append("../")

from utils.parse_arxiv import *
from keyword_extractor import *
from workloads.query_gen import *

In [2]:
file = open('../data/filtered_data.pickle', 'rb')
data = pickle.load(file)
file.close()

data.reset_index(drop=True, inplace=True)

# data.head(3)

In [3]:
save_path = None
k = 1
paper_num = 2
num_queries_per_paper = 10

In [4]:
file = open('../data/filtered_data.pickle', 'rb')
data = pickle.load(file)
file.close()
data.reset_index(drop=True, inplace=True)

# sample rows from the data
samples = data.sample(paper_num)
# sample = data.sample(1).iloc[0]
samples

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,cat_freq,journal_freq,date_freq
11629,2303.08207,Arslan Chaudhry,"Jiefeng Chen, Timothy Nguyen, Dilan Gorur, Ars...",Is forgetting less a good inductive bias for f...,Published as a conference paper at ICLR 2023,ICLR 2023,None,None,cs.LG cs.AI,http://creativecommons.org/licenses/by/4.0/,One of the main motivations of studying cont...,"[{'version': 'v1', 'created': 'Tue, 14 Mar 202...",2023-03-16,"[[Chen, Jiefeng, ], [Nguyen, Timothy, ], [Goru...",6910,17.0,747
5508,1911.04417,Agisilaos Chartsias,"Agisilaos Chartsias, Giorgos Papanastasiou, Ch...","Disentangle, align and fuse for multimodal and...",None,IEEE Transactions on Medical Imaging (2020),10.1109/TMI.2020.3036584,None,cs.CV,http://arxiv.org/licenses/nonexclusive-distrib...,Magnetic resonance (MR) protocols rely on se...,"[{'version': 'v1', 'created': 'Mon, 11 Nov 201...",2020-11-11,"[[Chartsias, Agisilaos, ], [Papanastasiou, Gio...",50512,4.0,993


In [6]:
sample_dict_list = []
for index, row in samples.iterrows():
    one_sample = dict(row)
    sample_dict_list.append(one_sample)
    
paper_id_list = []
queries_list = []
for d in sample_dict_list:
    query_template = QueryTemplate()
    query_template.parse_info(one_sample)
    queries_list.extend(query_template.generate_queries(num=num_queries_per_paper))
    paper_id_list.extend([d['id']]*num_queries_per_paper)

In [21]:
for i, q in enumerate(queries_list):
    # print("{}; {}. : {}".format(paper_id_list[i], i+1, q))
    # format print "query_num. id: query" fix space
    print("{:8d}. {:10}: {}".format(i+1, paper_id_list[i], q))

       1. 2303.08207: find papers written by < ScottSemple > about < t2 abdominal segmentation > published at < IEEE Transactions on Medical Imaging (2020) >
       2. 2303.08207: find papers from year 2020 on < cs.CV > about < blood oxygenation level dependent > published at < IEEE Transactions on Medical Imaging (2020) >
       3. 2303.08207: find papers written by < DavidE.Newby > from year 2020 published at < IEEE Transactions on Medical Imaging (2020) >
       4. 2303.08207: find papers from year 2020 on < cs.CV > published at < IEEE Transactions on Medical Imaging (2020) >
       5. 2303.08207: find papers on < cs.CV > about < accurate segmentation masks > published at < IEEE Transactions on Medical Imaging (2020) >
       6. 2303.08207: find papers from year 2020 on < cs.CV > published at < IEEE Transactions on Medical Imaging (2020) >
       7. 2303.08207: find papers from year 2020 on < cs.CV > published at < IEEE Transactions on Medical Imaging (2020) >
       8. 2303.08207: 